In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
from pathlib import Path
import pandas as pd
import geopandas as gpd
import numpy as np
from tqdm.notebook import tqdm
import sys
sys.path.append('/workspaces/my_dissertation')
from scripts.ais_parsers import discharge_to_csv, level_to_csv

In [3]:
russia_ws = gpd.read_file('../geo_data/geometry/russia_ws.gpkg')
russia_ws = russia_ws.set_index('gauge_id')
russia_ws

,name_ru,name_en,new_area,ais_dif,geometry
gauge_id,,,,,
9002,р.Енисей - г.Кызыл,r.Enisej - g.Kyzyl,115263.989,0.230,"POLYGON ((96.878 53.728, 96.878 53.727, 96.88 ..."
9022,р.Енисей - пос.Никитино,r.Enisej - pos.Nikitino,184499.118,1.373,"POLYGON ((96.878 53.727, 96.88 53.727, 96.88 5..."
9053,р.Енисей - пос.Базаиха,r.Enisej - pos.Bazaiha,302690.417,0.897,"POLYGON ((92.383 56.11, 92.383 56.11, 92.384 5..."
9112,р.Большой Енисей - с.Тоора-Хем,r.Bol'shoj Enisej - s.Toora-Hem,15448.834,0.969,"POLYGON ((97.071 52.716, 97.071 52.715, 97.074..."
9115,р.Большой Енисей - пос.Сейба,r.Bol'shoj Enisej - pos.Sejba,44934.891,0.751,"POLYGON ((96.878 53.727, 96.88 53.727, 96.88 5..."
...,...,...,...,...,...
11056,р.Иртыш - г.Тобольск,r.Irtysh - g.Tobol'sk,1540451.386,2.697,"POLYGON ((65.281 59.372, 65.281 59.371, 65.282..."
11061,р.Иртыш - г.Ханты-Мансийск,r.Irtysh - g.Hanty-Mansijsk,1694019.131,2.668,"POLYGON ((62.664 61.805, 62.665 61.805, 62.665..."
11801,р.Обь - г.Салехард,r.Ob' - g.Salehard,3004250.818,1.839,"POLYGON ((66.69 66.429, 66.69 66.428, 66.691 6..."


### From .xls to .csv

In [ ]:
q_path = glob.glob("/app/geo_data/ais_xls/discharge_xls/*.xls")

q_lbl_asso = list()
for i, f_path in enumerate(tqdm(q_path)):
    print(f"{f_path}, -- {i+1}")
    q_name = f_path.split("/")[-1][:-4]
    q_lbl_asso.append(discharge_to_csv(data_path=f_path, save_path=Path(f"./data_2/q_csv/{q_name}")))


In [4]:
h_path = glob.glob("/app/geo_data/ais_xls/levels_xls/*.xls")

id_lbl_asso = list()

for f_path in tqdm(h_path):
    h_name = f_path.split("/")[-1][:-4]
    id_lbl_asso.append(
        level_to_csv(
            data_path=f_path, save_path=Path(f"/app/geo_data/LvL_Discharge_2024/Level/{h_name}")
        )
    )

temp_df = list()
for record in id_lbl_asso:
    temp_df.append(pd.DataFrame(record).T)
temp_df = pd.concat(temp_df)
temp_df = temp_df.rename(columns={0: "name", 1: "height"})
temp_df.to_csv("/app/geo_data/LvL_Discharge_2024/height_id.csv")
gauge_height_file = pd.DataFrame()
for gauge_id in temp_df.index:
    temp_height = temp_df.loc[gauge_id, "height"].max()
    temp_name = temp_df.loc[gauge_id, "name"]
    if isinstance(temp_name, str):
        pass
    else:
        temp_name = temp_df.loc[gauge_id, "name"].drop_duplicates().to_list()[0]

    gauge_height_file.loc[str(gauge_id), ["height"]] = temp_height
    gauge_height_file.loc[str(gauge_id), ["name"]] = temp_name
    if str(gauge_id) in gauge_height_file.index:
        continue
gauge_height_file.index.name = "gauge_id"
gauge_height_file.to_csv("/app/geo_data/LvL_Discharge_2024/height_id.csv")


  0%|          | 0/309 [00:00<?, ?it/s]

# Parsing

### From by year to time series 

In [42]:
def ais_reader(file_path: str):
    data = pd.read_csv(file_path)
    if "Unnamed: 0" in data.columns:
        data = data.drop("Unnamed: 0", axis=1)
    data["date"] = pd.to_datetime(data["date"])
    data = data.set_index("date")

    return data


def file_by_district(all_files: list, save_storage: str, variable_tag: str, height_bs: pd.DataFrame):
    lvl_dict = {}
    for gauge in height_bs.index:
        lvl_dict[gauge] = []
        for lvl_f in all_files:
            if gauge == lvl_f.split("/")[-1][:-4]:
                lvl_dict[gauge].append(lvl_f)
    Path(f"{save_storage}/{variable_tag}").mkdir(parents=True, exist_ok=True)
    for gauge, files in lvl_dict.items():
        try:
            res_df = pd.concat([ais_reader(file) for file in files]).sort_index()
            res_df.to_csv(f"{save_storage}/{variable_tag}/{gauge}.csv")
        except ValueError:
            continue


#### Level

In [43]:
file_by_district(
    all_files=glob.glob("/app/geo_data/LvL_Discharge_2024/Level/*/*.csv"),
    height_bs=gauge_height_file,
    save_storage="/app/geo_data/LvL_Discharge_2024/UnifiedResults",
    variable_tag="h",
)


#### Discharge

In [44]:
file_by_district(
    all_files=glob.glob("/app/geo_data/LvL_Discharge_2024/Discharge/*/*.csv"),
    height_bs=gauge_height_file,
    save_storage="/app/geo_data/LvL_Discharge_2024/UnifiedResults",
    variable_tag="q",
)


#### All together

In [45]:
level_storage = "/app/geo_data/LvL_Discharge_2024/UnifiedResults/h"
discharge_storage = "/app/geo_data/LvL_Discharge_2024/UnifiedResults/q"
res_storage = "/app/geo_data/LvL_Discharge_2024/UnifiedResults/q_h"
Path(res_storage).mkdir(parents=True, exist_ok=True)

In [46]:
for gauge in gauge_height_file.index:
    lvl_file = pd.read_csv(f"{level_storage}/{gauge}.csv", index_col="date")
    lvl_file.index = pd.to_datetime(lvl_file.index)
    lvl_bs = gauge_height_file.loc[gauge, "height"]
    lvl_file["lvl_mbs"] = lvl_bs + lvl_file["level"] * 1e-2  # type: ignore
    lvl_file = lvl_file.rename(columns={"level": "lvl_sm"})
    try:
        dis_file = pd.read_csv(f"{discharge_storage}/{gauge}.csv", index_col="date")
        dis_file.index = pd.to_datetime(dis_file.index)
        try:
            ws_area = russia_ws.loc[f"{gauge}", "new_area"]
        except KeyError:
            print(f"Для поста {gauge} нет площади ):")
            ws_area = np.nan
        dis_file["q_mm_day"] = (dis_file["discharge"] * 86400) / (ws_area * 1e3)
        dis_file = dis_file.rename(columns={"discharge": "q_cms_s"})
        res_file = pd.concat([dis_file, lvl_file], axis=1)
        res_file = res_file[["lvl_sm", "q_cms_s", "lvl_mbs", "q_mm_day"]]
    except Exception:
        dis_file = pd.DataFrame()
        dis_file.index = pd.date_range(start="01/01/2008", end="12/31/2022")
        dis_file["date"] = pd.date_range(start="01/01/2008", end="12/31/2022")
        dis_file = dis_file.set_index("date")
        dis_file["q_cms_s"] = np.nan
        dis_file["q_mm_day"] = np.nan
        res_file = pd.concat([dis_file, lvl_file], axis=1)
        res_file = res_file[["lvl_sm", "q_cms_s", "lvl_mbs", "q_mm_day"]]

    res_file.to_csv(f"{res_storage}/{gauge}.csv")


Для поста 6529 нет площади ):
Для поста 6183 нет площади ):
Для поста 2253 нет площади ):
Для поста 2278 нет площади ):
Для поста 2280 нет площади ):
Для поста 7042 нет площади ):
Для поста 72590 нет площади ):
Для поста 72740 нет площади ):
Для поста 74804 нет площади ):
Для поста 74422 нет площади ):
Для поста 78638 нет площади ):
Для поста 78453 нет площади ):
Для поста 78456 нет площади ):
Для поста 78458 нет площади ):
Для поста 78468 нет площади ):
Для поста 83026 нет площади ):
Для поста 83028 нет площади ):
Для поста 83036 нет площади ):
Для поста 83040 нет площади ):
Для поста 83045 нет площади ):
Для поста 83048 нет площади ):
Для поста 83050 нет площади ):
Для поста 83056 нет площади ):
Для поста 83060 нет площади ):
Для поста 83068 нет площади ):
Для поста 83074 нет площади ):
Для поста 83083 нет площади ):
Для поста 83478 нет площади ):
Для поста 70638 нет площади ):
Для поста 9582 нет площади ):
Для поста 11520 нет площади ):
Для поста 12422 нет площади ):
Для поста 76133